In [ ]:
import os
import sys
from pathlib import Path

# patch python path with root repo directory
sys.path.append(str(Path(os.getcwd()).parent))

import base64
import io
import math

from IPython.display import HTML
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from src.index.storage import VectorIndex
from src.image.storage import LocalImageStorage
from src.index.builder import FaceNetPyTorchImageVectoriser

In [ ]:
plt.rcParams['figure.figsize'] = [16, 8]

In [ ]:
storage_path = "/Users/hayden.jeune/.celebstore"
image_storage = LocalImageStorage(storage_path + "/images")
index = VectorIndex(storage_path + "/vec")
vectoriser = FaceNetPyTorchImageVectoriser()

In [ ]:
def similarity(a: np.ndarray, b: np.ndarray) -> float:
    # just a dot product scaled between 0 and 1 for now
    return (np.dot(a, b) + 1) / 2

def get_lookalikes(image: Image.Image, n=5):
    me_vec = vectoriser.vectorise(image)

    def get_similarity(items) -> float:
        vec = items[1]
        return similarity(me_vec, vec)

    # need to sort out the dependency on the internals of
    items = sorted(index._vectors.items(), key=get_similarity, reverse=True)
    
    return [(name, image_storage.retrieve(name, "0"), similarity(me_vec, vec)) for name, vec in items[:n]]

In [ ]:
# HUGE thanks to https://colbrydi.github.io/accessing-a-computers-camera-inside-of-jupyter-without-installing-opencv-also-works-in-jupyterhub.html
# for this code snippet


from PIL import Image


main_text = """
<video id="video" width="560" height="420" autoplay></video>
<button id="snap">Snap Photo</button>
<canvas id="canvas" width="560" height="420"></canvas>

<script>
// Grab elements, create settings, etc.
var video = document.getElementById('video');

// Get access to the camera!
if(navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
    // Not adding `{ audio: true }` since we only want video now
    navigator.mediaDevices.getUserMedia({ video: true }).then(function(stream) {
        //video.src = window.URL.createObjectURL(stream);
        //video.play();
        video.srcObject=stream;
        video.play();
    });
}

// Elements for taking the snapshot
var canvas = document.getElementById('canvas');
var context = canvas.getContext('2d');
var video = document.getElementById('video');

// Trigger photo take
document.getElementById("snap").addEventListener("click", function() {
	context.drawImage(video, 0, 0, 560, 420);
    var myCanvas = document.getElementById('canvas');
    var image = myCanvas.toDataURL("image/png");
    IPython.notebook.kernel.execute("print('testing')")
    IPython.notebook.kernel.execute("image = '" + image + "'")
});
</script>

"""

def get_current_image() -> Image.Image:
    return Image.open(io.BytesIO(base64.b64decode(image.split(',')[1]))).convert("RGB")

HTML(main_text)

In [ ]:
user_image = get_current_image()
results = get_lookalikes(user_image, n=1)
name, img, sim = results[0]

fig, axs = plt.subplots(1, 2)

axs[0].axis("off")
axs[0].imshow(user_image)

axs[1].axis("off")
axs[1].set_title(f"{name} ({sim:.2f})")
axs[1].imshow(img)

In [ ]:
user_image = get_current_image()
results = get_lookalikes(user_image, n=4)

fig=plt.figure(figsize=(16,8))
grid = plt.GridSpec(2, 4, wspace=0, hspace=0.1)

ax = plt.subplot(grid[0:2, 0:2])
ax.axis("off")
ax.imshow(user_image)

start_col = 2
width = 2
height = 2

assert len(results) <= width * height

for i, (name, image, sim) in enumerate(results):
    col = start_col + i % width
    row = math.floor(i / width)
    
    ax = plt.subplot(grid[row, col])
    ax.axis("off")
    ax.set_title(name)
    ax.imshow(image)